In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from scipy.stats import pearsonr, spearmanr
from transformers import AutoTokenizer, EsmModel
import time
import lightning as L
import os
import scipy
import scipy.stats
import sklearn.metrics as skmetrics
import matplotlib.pyplot as plt
import seaborn as sns
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [ ]:
class ProtEmbeddingDiffDataset(Dataset):
    def __init__(self, tensor_folder, csv_file):
        self.df = pd.read_csv(csv_file)
        
        # build wt lookup: WT_name → wt embedding filename
        wt_df = self.df[self.df.mut_type == 'wt']
        wt_lookup = dict(zip(wt_df['WT_name'], wt_df['name']))
        
        # only keep mutation rows
        self.df = self.df[self.df.mut_type != 'wt'].reset_index(drop=True)
        self.labels   = torch.tensor(self.df['ddG_ML'].values)

        # preload all embeddings into RAM
        print("Loading embeddings into RAM...")
        self.embeddings = []
        for idx, row in self.df.iterrows():
            # mutant embedding
            mut_emb = torch.load(os.path.join(tensor_folder, row['name'] + '.pt'))['mean_representations'][6]
            # wildtype embedding
            wt_emb  = torch.load(os.path.join(tensor_folder, wt_lookup[row['WT_name']] + '.pt'))['mean_representations'][6]
            # difference
            diff = mut_emb - wt_emb
            # concatenate → (2304,)
            self.embeddings.append(torch.cat([mut_emb, wt_emb, diff], dim=0))
        print("Done!")

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx].float()

In [23]:
dataset_train = ProtEmbeddingDiffDataset(
    'data/mega_train_embeddings',
    'data/mega_train.csv'
)
dataset_val = ProtEmbeddingDiffDataset(
    'data/mega_val_embeddings',
    'data/mega_val.csv'
)

loader_train = DataLoader(dataset_train, batch_size=1024, shuffle=True,  num_workers=4)
loader_val   = DataLoader(dataset_val,   batch_size=512,  shuffle=False, num_workers=4)

In [30]:

from torchmetrics.regression import PearsonCorrCoef, SpearmanCorrCoef

class StabModel(L.LightningModule):
    def __init__(self, input_dim=768, lr=1e-2):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.Sigmoid(),
         
            nn.Linear(256, 1)
        )
        self.loss_fn = nn.MSELoss()
        self.lr = lr

        # Metrics
        self.val_pearson = PearsonCorrCoef()
        self.val_spearman = SpearmanCorrCoef()

    def forward(self, x):
        return self.model(x).squeeze(1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = self.loss_fn(preds, y)
        
        # Print Train Loss to console/progress bar
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = self.loss_fn(preds, y)
        
        # Update correlation metrics
        self.val_pearson(preds, y)
        self.val_spearman(preds, y)
        
        # Print Val stats to console/progress bar
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        self.log("val_pear", self.val_pearson, on_epoch=True, prog_bar=True)
        self.log("val_spear", self.val_spearman, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

In [ ]:
model = StabModel(lr = 1e-3)

trainer = L.Trainer(devices =1, max_epochs= 5)
trainer.fit(model, loader_train, loader_val)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params | Mode  | FLOPs
------------------------------------------------------------------
0 | model        | Sequential       | 197 K  | train | 0    
1 | loss_fn      | MSELoss          | 0      | train | 0    
2 | val_pearson  | PearsonCorrCoef  | 0      | train | 0    
3 | val_spearman | SpearmanCorrCoef | 0      | train | 0    
------------------------------------------------------------------
197

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs
